In [1]:
# project is to pull location data for all artists featured on rockfort.

# 1. dataframe with playlist items per row.
# 2. match with bandcamp embeds, or make links elsewhere.
# 3. add location data to dataframe.
# 4. analysis, only two axes, lat/long and mix number.

import pandas
import requests
import time
from requests_html import HTMLSession
from requests_html import HTML

dataframe = pandas.DataFrame(columns=['page'])
session = HTMLSession()

for x in ['']+[f'?page={i}' for i in range(2, 27)]:
    time.sleep(1)
    r = session.get(f'https://thequietus.com/features/rockfort{x}')
    if r.status_code == 200:
        for s in r.text.split('holder">')[1:]:
            s = HTML(html=s.split('<img')[0])
            dataframe.loc[len(dataframe)] = [(list(s.links)[0])]

print(len(dataframe))
dataframe.head()

73


,page
0,/articles/31236-jazzy-bazz-quinzequinze-review
1,/articles/30836-ziak-aquaserge-elg-review
2,/articles/30509-mhd-laylow-ateyaba-french-review
3,/articles/30198-mathieu-boogaerts-jac-berrocal...
4,/articles/29982-sourdure-gazo-heimat-sch-frenc...


In [2]:
def extract_text(row):
    time.sleep(1)
    r = requests.get(str(f"https://thequietus.com{row['page']}"))
    if r.status_code != 200:
        raise Exception(f"{row['page']} not found.")
    else:
        return r.text

dataframe['text'] = dataframe.apply(extract_text, axis=1)

print(len(dataframe))
dataframe.head()

73


,page,text
0,/articles/31236-jazzy-bazz-quinzequinze-review,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
1,/articles/30836-ziak-aquaserge-elg-review,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
2,/articles/30509-mhd-laylow-ateyaba-french-review,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
3,/articles/30198-mathieu-boogaerts-jac-berrocal...,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
4,/articles/29982-sourdure-gazo-heimat-sch-frenc...,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."


In [16]:
test = dataframe.copy()

def mix_numb(row):

    # manual override some source issues.

    if '28486-black-devil-disco-club' in row['page']:
        return '22'
    elif '28773-french-music-review-triple-go' in row['page']:
        return '23'
    elif '29088-sabrina-bellaouel-laura' in row['page']:
        return '24'
    else:
        text = row['text']
        for a,b in [('<p><strong>Quietus Mix', '</strong>'), 
                    ('<p><strong>Rockfort Quietus Mix', '</strong>'), 
                    ('<p><strong>Rockfort! Mix', '</strong>'), 
                    ('<p><b>Rockfort Quietus Mix', '</b>')]:
            if a in text:
                return text.split(a)[1].split(b)[0].strip().split(' ')[0].replace(':','')

test['mix_numb'] = test.apply(mix_numb, axis=1)
test = test.loc[~test.mix_numb.isin([None])]

print(len(test))
test.head(40)

30


,page,text,mix_numb
0,/articles/31236-jazzy-bazz-quinzequinze-review,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",30
1,/articles/30836-ziak-aquaserge-elg-review,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",29
2,/articles/30509-mhd-laylow-ateyaba-french-review,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",28
3,/articles/30198-mathieu-boogaerts-jac-berrocal...,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",27
4,/articles/29982-sourdure-gazo-heimat-sch-frenc...,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",26
5,/articles/29706-alpha-wann-emmanuelle-parrenin...,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",25
7,/articles/29088-sabrina-bellaouel-laura-perrud...,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",24
8,/articles/28773-french-music-review-triple-go-...,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",23
9,/articles/28486-black-devil-disco-club-felicia...,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",22
10,/articles/28144-christine-ott-satan-sylvain-ch...,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",21


In [ ]:
# two additional columns required, playlist block of data, and external media links (bandcamp, soundcloud, youtube)